***ANURAG***

Collaborative based filtering
1. User based approach (go through various techniques like SVD, etc.)


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
!pip install surprise
import surprise
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
from surprise.model_selection import GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader, Dataset, SVD, model_selection

In [3]:
ratings=pd.read_csv('/content/drive/My Drive/datasets/Ratings .csv')
movies=pd.read_csv('/content/drive/My Drive/datasets/movies.csv')
userdata=pd.read_csv('/content/drive/My Drive/datasets/user_data.csv')

In [ ]:
print('shape of ratings dataset ::',ratings.shape)
print('shape of movies dataset ::',movies.shape)
print('shape of userdata dataset ::',userdata.shape)

shape of ratings dataset :: (1000209, 4)
shape of movies dataset :: (3883, 3)
shape of userdata dataset :: (6040, 5)


In [4]:
#merge datasets:: ratings,movies and userdata
d=pd.merge(userdata,ratings,on='UserID',how='left')
df=pd.merge(d,movies,on='MovieID',how='left')

In [ ]:
split_data,split_data.Genres = df.iloc[[i   for i,j in enumerate(df['Genres']) for value in j.split("|")] , :],[value for i in df['Genres'] for value in i.split("|")]
split_data.head(10)

***Experiment with Surprise SVD***

In [5]:
d = pd.read_csv('/content/drive/My Drive/datasets/Ratings .csv')
reader = Reader()
data = Dataset.load_from_df(d[['UserID', 'MovieID', 'Rating']], reader)

In [ ]:
svd = SVD()
print("CROSS VALIDATING THE DATASET", '\n')
model_selection.cross_validate(svd, data, measures=['rmse'],cv=5, verbose=True)
print("DONE CROSS VALIDATING THE DATASET",'\n')

print("TRAINING THE MODEL")
trainset = data.build_full_trainset()
sd=svd.fit(trainset)
print("DONE TRAINING THE MODEL", '\n')

CROSS VALIDATING THE DATASET 

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8732  0.8718  0.8766  0.8744  0.8738  0.8740  0.0016  
Fit time          49.13   49.73   49.70   50.39   49.34   49.66   0.43    
Test time         1.80    1.65    1.67    1.70    1.67    1.70    0.05    
DONE CROSS VALIDATING THE DATASET 

TRAINING THE MODEL
DONE TRAINING THE MODEL 



In [ ]:
#predict rating
y=[ ]
for i in range(1,6040):
  for j in range(1,3700):
    y.append(sd.predict(uid=i, iid=j, r_ui=1))
len(y)

22338261

In [ ]:
e=pd.DataFrame(y)
e.head()

,uid,iid,r_ui,est,details
0,1,1,1,4.419962,{'was_impossible': False}
1,1,2,1,3.605446,{'was_impossible': False}
2,1,3,1,3.481897,{'was_impossible': False}
3,1,4,1,2.934002,{'was_impossible': False}
4,1,5,1,3.370183,{'was_impossible': False}


In [ ]:
trainset, testset = train_test_split(data, test_size =.25)
predictions = svd.fit(trainset).test(testset)

In [ ]:
accuracy.rmse(predictions)

RMSE: 0.8761


0.8761252657385575

In [ ]:
df1=pd.DataFrame(predictions)
df1.shape

(250053, 5)

In [ ]:
def recommend(userid_, n):
  f=e.loc[e[(e.uid==userid_)].est.sort_values(ascending=False).index][:n].iid.tolist()
  p=movies.loc[movies[movies.MovieID.isin(f)].index]
  k = ratings.loc[ratings[(ratings.UserID==userid_)].index]['MovieID'].tolist()
  return p.loc[p[~p.MovieID.isin(k)].index]
  #return p

In [ ]:
u=df.loc[df[(df.UserID==1)].index]['Genres']
u.mode()
u2=u1.loc[u1.index]['Genres']
u2.mode().iloc[0]

'Drama'

In [ ]:
recommend(1, 15)

,MovieID,Title,Genres
49,50,"Usual Suspects, The (1995)",Crime|Thriller
108,110,Braveheart (1995),Action|Drama|War
293,296,Pulp Fiction (1994),Crime|Drama
315,318,"Shawshank Redemption, The (1994)",Drama
589,593,"Silence of the Lambs, The (1991)",Drama|Thriller
643,649,Cold Fever (� k�ldum klaka) (1994),Comedy|Drama
941,953,It's a Wonderful Life (1946),Drama
1222,1242,Glory (1989),Action|Drama|War
1854,1923,There's Something About Mary (1998),Comedy
2255,2324,Life Is Beautiful (La Vita � bella) (1997),Comedy|Drama


In [ ]:

# def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    # top_n = defaultdict(list)
    # for uid, iid, true_r, est, _ in predictions:
        # top_n[uid].append((iid, est))
    # Then sort the predictions for each user and retrieve the k highest ones.
    # for uid, user_ratings in top_n.items():
        # user_ratings.sort(key=lambda x: x[1], reverse=True)
        # top_n[uid] = user_ratings[:n]
    # return top_n

In [ ]:
top_n = get_top_n(predictions, n=8)
#Print the recommended items for each user
print("PRINTING RECOMMENDATION")
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])



***USER BASED COLLABORATIVE USING NMF(Non-negative Matrix Factorization.) Surprise***

In [7]:
from surprise import NMF
nmf= NMF()
print("CROSS VALIDATING THE DATASET", '\n')
model_selection.cross_validate(nmf, data, measures=['rmse'],cv=5, verbose=True)
print("DONE CROSS VALIDATING THE DATASET",'\n')

print("TRAINING THE MODEL")
trainset = data.build_full_trainset()
nmf.fit(trainset)
print("DONE TRAINING THE MODEL", '\n')

CROSS VALIDATING THE DATASET 

Evaluating RMSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9191  0.9180  0.9149  0.9161  0.9154  0.9167  0.0016  
Fit time          55.69   56.28   56.11   55.54   55.17   55.76   0.40    
Test time         2.26    2.20    2.57    1.87    2.21    2.22    0.22    
DONE CROSS VALIDATING THE DATASET 

TRAINING THE MODEL
DONE TRAINING THE MODEL 



In [8]:
trainset, testset = train_test_split(data, test_size =.25)
predictions = nmf.fit(trainset).test(testset)

In [53]:
accuracy.rmse(predictions)

RMSE: 0.9171


0.9171131575150564

In [9]:
df1=pd.DataFrame(predictions)
df1.shape

(250053, 5)

In [ ]:
def recommend(userid_, n):
  f=e1.loc[e1[(e1.uid==userid_)].est.sort_values(ascending=False).index][:n].iid.tolist()
  p=movies.loc[movies[movies.MovieID.isin(f)].index]
  k = ratings.loc[ratings[(ratings.UserID==userid_)].index]['MovieID'].tolist()
  return p.loc[p[~p.MovieID.isin(k)].index]
  #return p

In [ ]:
y1=[]
for i in range(1,len(userdata)):
  for j in range(1,len(movies)):
    y1.append(nmf.predict(uid=i, iid=j, r_ui=1))
len(y1)

23443398

In [ ]:
e1=pd.DataFrame(y1)
e1.head()

In [ ]:
recommend(1,15)

,MovieID,Title,Genres
27,28,Persuasion (1995),Romance
28,29,"City of Lost Children, The (1995)",Adventure|Sci-Fi
29,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Drama
31,32,Twelve Monkeys (1995),Drama|Sci-Fi
36,37,Across the Sea of Time (1995),Documentary
40,41,Richard III (1995),Drama|War
46,47,Seven (Se7en) (1995),Crime|Thriller
48,49,When Night Is Falling (1995),Drama|Romance
49,50,"Usual Suspects, The (1995)",Crime|Thriller
52,53,Lamerica (1994),Drama


In [52]:
nmf.predict(uid=1, iid=1, r_ui=1)

Prediction(uid=1, iid=1, r_ui=1, est=4.157014663371732, details={'was_impossible': False})

****UBCF by TruncatedSVD scikit-learn***

In [10]:
pivot_table=df.pivot_table(index='Title',columns='UserID',values='Rating').fillna(0)
pivot_table.head()

UserID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"$1,000,000 Duck (1971)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Night Mother (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...And Justice for All (1979),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
pivot_table.shape

(3706, 6040)

In [13]:
pivot_t = pivot_table.values.T
pivot_t.shape

(6040, 3706)

In [15]:
import sklearn
from sklearn.decomposition import TruncatedSVD
model_svd = TruncatedSVD(n_components=5, n_iter=5, random_state=50)
matrix = model_svd.fit_transform(pivot_t)
matrix.shape

(6040, 5)

In [42]:
corr = np.corrcoef(matrix)
corr.shape

(6040, 6040)

In [43]:
_title = pivot_table.columns
_title_list = list(_title)
t_name = _title_list.index(15)
print(t_name)

14


In [44]:
corr_t

array([0.4809083 , 0.95176784, 0.65020386, ..., 0.35572462, 0.06907104,
       0.52918954])

In [ ]:
_title_list

In [ ]:
corr_t = corr[t_name]
_title[(corr_t)]

In [ ]:
list(corr)

In [48]:
corr_t  

array([0.4809083 , 0.95176784, 0.65020386, ..., 0.35572462, 0.06907104,
       0.52918954])

***Q. Math behind Matix Factrorization***

* **Model-based Collaborative Filtering** is based on matrix factorization (MF) mainly as an unsupervised learning method for latent variable decomposition and dimensionality reduction. Matrix factorization is widely used for recommender systems.
* **Matrix Factorization**
                             Rating Matrix[R] = User Matrix[P] * Item Matrix[Q](trans.)
                      R(m user * n Items) = P(m user * k features) * Q(k feature * n items)(trans.)
                                              
    * The goal of MF is to learn the latent preferences of users and the latent attributes of items from known ratings to then predict the unknown ratings through the dot product of the latent features of users and items.

    * When we have a sparse matrix, with a lot of dimensions, by doing matrix factorization, you can restructure the user-item matrix and you can represent the matrix by the multiplication of two matrices, where the rows contain the latent vector.

    * You fit this matrix to approximate your original matrix, as closely as possible, by multiplying the two matrices together, which fills in the entries missing in the original matrix.

* **Singular value decomposition (SVD)***
                                         R = U*S*V(transpose)
                              R(m*n) = U(m*m) * S(diagonal Matix)* V(n*n)(trans.)
    * SVD is the decomposition of a matrix A into 3 matrices – U, S, and V
      * Diagonal Value are called as **Singular Values**
      * U is an m x m matrix of Left Singular Vectors
      * S is an m x n rectangular diagonal matrix of Singular Values arranged in decreasing order
      * V is an n x n matrix of Right Singular Vectors 
      * If the dimensions of A are m x n:


